In [1]:
from datetime import datetime
start_time = datetime.now()

# Libraries to install
# %pip install pandas-profiling

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

print('LOAD DURATION: ', datetime.now() - start_time) # load time about 30 seconds

LOAD DURATION:  0:00:00.949757


In [2]:
data = pd.read_csv('NY property data.csv')

In [3]:
data.shape

(1070994, 32)

In [4]:
numrecords = len(data)

### remove benign properties

In [5]:
# first look at the most frequent owners
remove_list=data['OWNER'].value_counts().head(20).index.tolist()

In [6]:
remove_list

['PARKCHESTER PRESERVAT',
 'PARKS AND RECREATION',
 'DCAS',
 'HOUSING PRESERVATION',
 'CITY OF NEW YORK',
 'DEPT OF ENVIRONMENTAL',
 'BOARD OF EDUCATION',
 'NEW YORK CITY HOUSING',
 'CNY/NYCTA',
 'NYC HOUSING PARTNERSH',
 'YORKVILLE TOWERS ASSO',
 'DEPARTMENT OF BUSINES',
 'DEPT OF TRANSPORTATIO',
 'MTA/LIRR',
 'PARCKHESTER PRESERVAT',
 'MH RESIDENTIAL 1, LLC',
 '434 M LLC',
 'LINCOLN PLAZA ASSOCIA',
 'DEUTSCHE BANK NATIONA',
 '561 11TH AVENUE TMG L']

In [7]:
# remove some of these & add some others to also be removed 
remove_list.remove('YORKVILLE TOWERS ASSO')
remove_list.remove('434 M LLC')
remove_list.remove('DEUTSCHE BANK NATIONA')
remove_list.remove('561 11TH AVENUE TMG L')

In [8]:
remove_list.append('UNITED STATES OF AMER')
remove_list.append('U S GOVERNMENT OWNRD')
remove_list.append('THE CITY OF NEW YORK')
remove_list.append('NYS URBAN DEVELOPMENT')
remove_list.append('NYS DEPT OF ENVIRONME')
remove_list.append('CULTURAL AFFAIRS')
remove_list.append('DEPT OF GENERAL SERVI')
remove_list.append('DEPT RE-CITY OF NY')

In [9]:
prop_data=data[~data['OWNER'].isin(remove_list)].reset_index(drop=True)
prop_data.shape

(1046826, 32)

In [10]:
data.shape

(1070994, 32)

In [11]:
# number of records removed
len(data)-len(prop_data)

24168

In [15]:
# replace NAN zip codes with 0 and count
missing_zips=np.where(pd.isnull(prop_data['ZIP']))[0]
len(missing_zips)

21772

In [16]:
# Assume the data is sorted by zip code.
# fill missing zip code with before/after zip code, if they are the same
for i in range(len(missing_zips)):
    if(prop_data.loc[missing_zips[i]+1,'ZIP'])==prop_data.loc[missing_zips[i]-1,'ZIP']:
        prop_data.loc[missing_zips[i],'ZIP']=prop_data.loc[missing_zips[i]-1,'ZIP']

In [17]:
# how many are still left to fill?
missing_zips=np.where(pd.isnull(prop_data['ZIP']))[0]
len(missing_zips)

10245

In [18]:
# for the remaining missing zips,
# fill in the previous record's zip
for i in range(len(missing_zips)):
    prop_data.loc[missing_zips[i],'ZIP']=prop_data.loc[missing_zips[i]-1,'ZIP']

In [19]:
missing_zips=np.where(pd.isnull(prop_data['ZIP']))[0]
len(missing_zips)

0

In [20]:
# fill in missing stories
temp=prop_data[prop_data['STORIES'].isnull()]
len(temp)

43968

In [22]:
temp['TAXCLASS'].value_counts()

1B    19055
4     17979
2      3434
3      2416
1       877
2C      138
2B       34
2A       30
1A        5
Name: TAXCLASS, dtype: int64

In [23]:
mean_stories=prop_data.groupby('TAXCLASS')['STORIES'].mean()
print(mean_stories)

TAXCLASS
1      2.111633
1A     1.656867
1B     4.000000
1C     3.052748
1D     1.068966
2     16.310549
2A     2.844032
2B     4.001635
2C     4.744977
3      1.000000
4      5.517257
Name: STORIES, dtype: float64


In [24]:
# fill in with the avg by tax class
len(prop_data[prop_data['STORIES']==0])

0

In [25]:
prop_data['STORIES']=prop_data['STORIES'].fillna(value=0)
for i in mean_stories.index:
    prop_data.loc[(prop_data['STORIES']==0)&(prop_data['TAXCLASS']==i),'STORIES']=mean_stories[i]

In [27]:
# calculate means for AVTOT,AVLAND, FULLVAL by taxclass
prop_data['FULLVAL'].replace('NaN',0)
temp=prop_data[prop_data['FULLVAL']!=0]
mean_fullval=temp.groupby('TAXCLASS')['FULLVAL'].mean()
print(mean_fullval)

TAXCLASS
1     5.700059e+05
1A    3.352942e+05
1B    3.738399e+05
1C    7.615359e+05
1D    2.233614e+07
2     8.005832e+05
2A    8.639066e+05
2B    1.254525e+06
2C    7.723493e+05
3     8.510005e+04
4     2.772747e+06
Name: FULLVAL, dtype: float64


In [28]:
prop_data['AVLAND'].replace('NaN',0)
temp_avland=prop_data[prop_data['AVLAND']!=0]
mean_avland=temp_avland.groupby('TAXCLASS')['AVLAND'].mean()
print(mean_avland)

TAXCLASS
1      14901.860425
1A      2247.472465
1B      9185.394750
1C      8225.658898
1D    709303.793103
2      88471.102578
2A     31370.740197
2B     54607.394911
2C     25635.944099
3      28135.636364
4     444996.049088
Name: AVLAND, dtype: float64


In [29]:
prop_data['AVTOT'].replace('NaN',0)
temp_avtot=prop_data[prop_data['AVTOT']!=0]
mean_avtot=temp_avtot.groupby('TAXCLASS')['AVTOT'].mean()
print(mean_avtot)

TAXCLASS
1     2.502549e+04
1A    1.437036e+04
1B    9.189080e+03
1C    2.898447e+04
1D    1.166866e+06
2     3.602638e+05
2A    7.963525e+04
2B    1.790961e+05
2C    1.170418e+05
3     3.829505e+04
4     1.294237e+06
Name: AVTOT, dtype: float64


In [30]:
# fill in with the avg by tax class
for i in mean_fullval.index:
    prop_data.loc[(prop_data['FULLVAL']==0)&(prop_data['TAXCLASS']==i),'FULLVAL']=mean_fullval[i]
    prop_data.loc[(prop_data['AVLAND']==0)&(prop_data['TAXCLASS']==i),'AVLAND']=mean_avland[i]
    prop_data.loc[(prop_data['AVTOT']==0)&(prop_data['TAXCLASS']==i),'AVTOT']=mean_avtot[i]

In [31]:
# Because these 4 fields do not have NAs, 
# we just need to replace 0s and the 1s. 
# We think zero and 1 are both invalid values for these fields, 
# so replace them with NA.
# Calculate groupwise average. Replace 0 and 1's by NAs so they are not counted in calculating mean.
prop_data.loc[prop_data['LTFRONT']==0,'LTFRONT']=np.nan
prop_data.loc[prop_data['LTDEPTH']==0,'LTDEPTH']=np.nan
prop_data.loc[prop_data['BLDFRONT']==0,'BLDFRONT']=np.nan
prop_data.loc[prop_data['BLDDEPTH']==0,'BLDDEPTH']=np.nan
prop_data.loc[prop_data['LTFRONT']==1,'LTFRONT']=np.nan
prop_data.loc[prop_data['LTDEPTH']==1,'LTDEPTH']=np.nan
prop_data.loc[prop_data['BLDFRONT']==1,'BLDFRONT']=np.nan
prop_data.loc[prop_data['BLDDEPTH']==1,'BLDDEPTH']=np.nan

In [32]:
# Calculate mean now (mean function ignores NAs but not 0s hence we converted 0 to NA)
mean_LTFRONT=prop_data.groupby(prop_data['TAXCLASS'])['LTFRONT'].mean()
mean_LTDEPTH=prop_data.groupby(prop_data['TAXCLASS'])['LTDEPTH'].mean()
mean_BLDFRONT=prop_data.groupby(prop_data['TAXCLASS'])['BLDFRONT'].mean()
mean_BLDDEPTH=prop_data.groupby(prop_data['TAXCLASS'])['BLDDEPTH'].mean()

# Now impute values with the average for that record's TAXCLASS
for index in mean_LTFRONT.index:
    prop_data.loc[(prop_data['LTFRONT'].isnull())&(prop_data['TAXCLASS']==index),'LTFRONT']=mean_LTFRONT[index]    
    prop_data.loc[(prop_data['LTDEPTH'].isnull())&(prop_data['TAXCLASS']==index),'LTDEPTH']=mean_LTDEPTH[index]
    prop_data.loc[(prop_data['BLDFRONT'].isnull())&(prop_data['TAXCLASS']==index),'BLDFRONT']=mean_BLDFRONT[index]
    prop_data.loc[(prop_data['BLDDEPTH'].isnull())&(prop_data['TAXCLASS']==index),'BLDDEPTH']=mean_BLDDEPTH[index]

In [52]:
data=prop_data.copy()
data.shape

(1046826, 32)

In [53]:
# convert ZIP to a str rather than a float
# we will call the 1st 3 digits of the zip code zip3
data['ZIP']=data['ZIP'].astype(str)
data['zip3']=data['ZIP'].str[:3]

In [37]:
data['zip3']

0          100
1          100
2          100
3          100
4          100
          ... 
1046821    103
1046822    103
1046823    103
1046824    103
1046825    103
Name: zip3, Length: 1046826, dtype: object

In [54]:
data['ltsize']=data['LTFRONT']*data['LTDEPTH']
data['bldsize']=data['BLDFRONT']*data['BLDDEPTH']
data['bldvol']=data['bldsize']*data['STORIES']

In [55]:
data['r1']=data['FULLVAL']/data['ltsize']
data['r2']=data['FULLVAL']/data['bldsize']
data['r3']=data['FULLVAL']/data['bldvol']
data['r4']=data['AVLAND']/data['ltsize']
data['r5']=data['AVLAND']/data['bldsize']
data['r6']=data['AVLAND']/data['bldvol']
data['r7']=data['AVTOT']/data['ltsize']
data['r8']=data['AVTOT']/data['bldsize']
data['r9']=data['AVTOT']/data['bldvol']

In [56]:
nine_vars=['r1','r2','r3','r4','r5','r6','r7','r8','r9']
zip5_mean=data.groupby('ZIP')[nine_vars].mean()
zip3_mean=data.groupby('zip3')[nine_vars].mean()
taxclass_mean=data.groupby('TAXCLASS')[nine_vars].mean()
boro_mean=data.groupby('BORO')[nine_vars].mean()

In [59]:
boro_mean

,r1,r2,r3,r4,r5,r6,r7,r8,r9
BORO,,,,,,,,,
1,359.235203,517.942490,111.912748,33.469383,47.494151,10.445557,92.859664,128.007681,21.781626
2,143.617634,410.467056,202.649005,6.016193,17.698662,10.120771,16.350790,39.693424,19.457085
3,244.903151,581.772351,264.057924,6.886018,16.380294,7.899103,17.642275,37.070493,16.246135
4,168.541827,563.936096,287.130052,6.158232,21.636150,11.762399,13.203304,41.022134,21.017958
5,152.528063,467.705704,255.387365,8.924127,18.476265,10.096301,21.502513,31.731270,17.453270


In [60]:
data=data.join(zip5_mean,on='ZIP',rsuffix='_zip5')
data=data.join(zip3_mean,on='zip3',rsuffix='_zip3')
data=data.join(taxclass_mean,on='TAXCLASS',rsuffix='_taxclass')
data=data.join(boro_mean,on='BORO',rsuffix='_boro')
rsuffix=['_zip5','_zip3','_taxclass','_boro']
for var in nine_vars:
    for r in rsuffix:
        data[str(var)+r]=data[var]/data[str(var)+r]


In [61]:
data.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
RECORD,1046826.0,537122.108419,308087.821735,1.000000e+00,273156.250000,539082.500000,802736.750000,1.070994e+06
BORO,1046826.0,3.221716,1.198952,1.000000e+00,3.000000,3.000000,4.000000,5.000000e+00
BLOCK,1046826.0,4757.324964,3677.503964,1.000000e+00,1542.000000,4078.000000,6920.000000,1.635000e+04
LOT,1046826.0,349.840286,825.822800,1.000000e+00,23.000000,49.000000,139.000000,9.502000e+03
LTFRONT,1046826.0,50.318139,60.738367,2.000000e+00,21.000000,30.000000,55.000000,9.999000e+03
...,...,...,...,...,...,...,...,...
r5_boro,1046826.0,1.000000,17.146866,4.413439e-06,0.296494,0.774854,1.052615,1.546654e+04
r6_boro,1046826.0,1.000000,16.042330,9.937551e-07,0.216734,0.696596,1.000643,9.483223e+03
r7_boro,1046826.0,1.000000,7.276792,3.875516e-07,0.257662,0.514318,0.814198,2.407601e+03
r8_boro,1046826.0,1.000000,21.931794,8.187498e-06,0.414725,0.696414,0.929975,2.124127e+04


In [62]:
data.columns

Index(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol', 'r1', 'r2', 'r3', 'r4', 'r5', 'r6', 'r7', 'r8', 'r9',
       'r1_zip5', 'r2_zip5', 'r3_zip5', 'r4_zip5', 'r5_zip5', 'r6_zip5',
       'r7_zip5', 'r8_zip5', 'r9_zip5', 'r1_zip3', 'r2_zip3', 'r3_zip3',
       'r4_zip3', 'r5_zip3', 'r6_zip3', 'r7_zip3', 'r8_zip3', 'r9_zip3',
       'r1_taxclass', 'r2_taxclass', 'r3_taxclass', 'r4_taxclass',
       'r5_taxclass', 'r6_taxclass', 'r7_taxclass', 'r8_taxclass',
       'r9_taxclass', 'r1_boro', 'r2_boro', 'r3_boro', 'r4_boro', 'r5_boro',
       'r6_boro', 'r7_boro', 'r8_boro', 'r9_boro'],
      dtype='object')

In [63]:
data.drop(['RECORD', 'BBLE', 'BORO', 'BLOCK', 'LOT', 'EASEMENT', 'OWNER', 'BLDGCL',
       'TAXCLASS', 'LTFRONT', 'LTDEPTH', 'EXT', 'STORIES', 'FULLVAL', 'AVLAND',
       'AVTOT', 'EXLAND', 'EXTOT', 'EXCD1', 'STADDR', 'ZIP', 'EXMPTCL',
       'BLDFRONT', 'BLDDEPTH', 'AVLAND2', 'AVTOT2', 'EXLAND2', 'EXTOT2',
       'EXCD2', 'PERIOD', 'YEAR', 'VALTYPE', 'zip3', 'ltsize', 'bldsize',
       'bldvol'], axis=1, inplace=True)

In [66]:
stats=data.describe().transpose()

In [67]:
stats.to_excel('stats_on_vars.xlsx')